# **Avarich speech to text and text to speech Code**






This notebook contains two models built will be deployed in our mobile app to ensure vocale interaction with our financial  avatar advisor

1.   The first model is a Speech-to-Text model. We utilized Transformer's Speech2TextProcessor and Speech2TextForConditionalGeneration.



2. The third model is a Text-to-Speech model. It takes the output from the second model and transforms it into speech using the SpeechT5Processor, SpeechT5ForTextToSpeech, and SpeechT5HifiGan. Additionally, the set_seed function is implemented to ensure reproducibility in the model's results.

**All three models have been thoroughly tested and function well in the notebook**.

## **Speech To Text**

In [6]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
import librosa
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Change the working directory to the folder where your file is located
import os
os.chdir('/content/drive/MyDrive')

# Load the speech-to-text model and processor
model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

# Load your own audio file
audio_file_path = 'AVARICH.mp3'
# Adjust chunk size according to your preference and memory constraints
chunk_size_seconds = 10

# Function to transcribe audio chunks
def transcribe_audio_chunk(audio_chunk):
    inputs = processor(audio_chunk, sampling_rate=16000, return_tensors="pt")
    generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
    return transcription

# Load the audio file
audio_signal, sampling_rate = librosa.load(audio_file_path, sr=16000)

# Calculate the number of chunks
chunk_size_samples = chunk_size_seconds * sampling_rate
num_chunks = len(audio_signal) // chunk_size_samples

# Transcribe each chunk and concatenate the transcriptions into a string
transcription_string = ""
for i in range(num_chunks + 1):
    start = i * chunk_size_samples
    end = min((i + 1) * chunk_size_samples, len(audio_signal))
    audio_chunk = audio_signal[start:end]
    transcription_chunk = transcribe_audio_chunk(audio_chunk)
    transcription_string += ' '.join(transcription_chunk) + ' '

print("Transcription:", transcription_string)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.decoder.embed_positions.weights', 'model.encoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Transcription: what are the main aspects of personal finance 


##**Text To Speech Model**




In [4]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan, set_seed
import torch
import IPython.display as ipd

# Load the processor, model, and vocoder
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# Define the text to convert to speech
text_to_convert = "Personal finance involves managing your income, expenses, savings, investments, and debt to achieve financial stability."

# Process the text input
inputs = processor(text=text_to_convert, return_tensors="pt")

# Load speaker embeddings from an external source or create a placeholder
# Replace this with actual speaker embeddings
speaker_embeddings = torch.zeros((1, 512))  # Replace with actual speaker embeddings

# Set seed for reproducibility
set_seed(555)

# Generate the speech
try:
    speech = model.generate(inputs["input_ids"], speaker_embeddings=speaker_embeddings, vocoder=vocoder)

    # Convert the generated speech tensor to numpy array for playback
    audio_array = speech.squeeze().numpy()

    # Display the audio player for generated audio
    audio_player = ipd.Audio(audio_array, rate=22050)  # Adjust rate if necessary
    display(audio_player)  # This will display the audio player in the notebook

except ValueError as e:
    print(f"Error: {e}")
